#### Análise de Séries Temporais

#### Conteúdo - Bases e Notebook da aula

#### Importação de pacotes, bibliotecas e funções (def)

In [ ]:
# Importar biblioteca completa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 

# Importar função especifica de um módulo
from statsmodels.tsa.seasonal import seasonal_decompose

#### Tendencia, Sazonalidade e Residuo


Em análise de séries temporais, entender os componentes de **tendência, sazonalidade e resíduo** é fundamental para interpretar e prever o comportamento de dados ao longo do tempo. Uma série temporal é, essencialmente, um conjunto de dados coletados em intervalos de tempo sequenciais. A decomposição desses dados em seus componentes principais nos ajuda a compreender os padrões subjacentes.

Imagine o total de vendas mensais de uma sorveteria. Esse dado bruto, ao longo de vários anos, pode ser decomposto nos seguintes elementos:

### 1. Tendência ($T_t$)

A tendência representa o comportamento de longo prazo da série temporal. É a direção geral para a qual os dados estão se movendo, ignorando as flutuações de curto prazo. Uma tendência pode ser:

* **Crescente:** Quando os valores da série aumentam ao longo do tempo. Por exemplo, o aumento gradual das vendas de uma empresa ano após ano devido ao crescimento do mercado.
* **Decrescente:** Quando os valores da série diminuem ao longo do tempo. Um exemplo seria a diminuição do número de cartas enviadas pelos correios com a popularização do e-mail.
* **Estável (ou Horizontal):** Quando não há uma direção clara de aumento ou queda a longo prazo.

É importante notar que a tendência não precisa ser linear. Ela pode mudar de direção ao longo do tempo, como uma tendência de crescimento que se torna estável e depois decrescente.

> **Exemplo na sorveteria:** Uma tendência de crescimento nas vendas anuais pode indicar que a marca está se tornando mais popular ou que a população local está aumentando.

### 2. Sazonalidade ($S_t$)

A sazonalidade refere-se a padrões que se repetem em intervalos de tempo fixos e conhecidos. Essas flutuações são previsíveis e estão, geralmente, associadas a fatores sazonais, como o clima, feriados ou estações do ano. A sazonalidade pode ser:

* **Anual:** Padrões que se repetem todos os anos (ex: vendas de Natal).
* **Mensal:** Padrões que ocorrem em determinados dias do mês.
* **Semanal:** Padrões que se repetem ao longo da semana (ex: maior movimento em restaurantes nos fins de semana).
* **Diária:** Padrões que ocorrem em certas horas do dia (ex: picos de trânsito no horário de pico).

A principal característica da sazonalidade é que o período de repetição é fixo e conhecido.

> **Exemplo na sorveteria:** As vendas de sorvete são muito maiores durante o verão (dezembro a março no Hemisfério Sul) e caem drasticamente no inverno. Esse padrão se repete todos os anos, caracterizando a sazonalidade.

### 3. Resíduo ($R_t$ ou $\epsilon_t$)

O resíduo, também conhecido como ruído, erro ou componente aleatório, é o que sobra na série temporal após a remoção da tendência e da sazonalidade. Ele representa as flutuações irregulares e imprevisíveis nos dados. O resíduo pode ser causado por:

* Eventos inesperados e não recorrentes (ex: uma promoção relâmpago que aumentou as vendas em um dia específico).
* Variações aleatórias e naturais nos dados.
* Erros de medição.

Em um bom modelo de série temporal, o resíduo deve ser aleatório, sem nenhum padrão discernível, com média zero e variância constante. Se houver padrões no resíduo, isso indica que o modelo não capturou todas as informações importantes dos dados.

> **Exemplo na sorveteria:** Um aumento repentino nas vendas em um dia de outono, que não pode ser explicado pela tendência ou sazonalidade, seria parte do resíduo. Talvez tenha sido um dia atipicamente quente e ensolarado.

### Decomposição da Série Temporal

A relação entre esses três componentes pode ser modelada de duas formas principais:

* **Modelo Aditivo:**
    $$
    Y_t = T_t + S_t + R_t
    $$
    Neste modelo, a magnitude da sazonalidade e do resíduo não depende do nível da tendência. É adequado para séries em que as flutuações sazonais são relativamente constantes ao longo do tempo.

* **Modelo Multiplicativo:**
    $$
    Y_t = T_t \times S_t \times R_t
    $$
    Aqui, a magnitude da sazonalidade e do resíduo é proporcional ao nível da tendência. Ou seja, as flutuações sazonais aumentam ou diminuem conforme a tendência sobe ou desce. Este modelo é frequentemente mais realista para séries econômicas e de vendas.

A análise desses componentes é crucial para a construção de modelos de previsão precisos, pois permite que cada padrão seja modelado separadamente, levando a uma melhor compreensão do fenômeno observado.

#### Aula 1 - O Problema de Time Series

In [ ]:
# Atribuir em uma variavel o link do conteudo
df_path = "https://raw.githubusercontent.com/carlosfab/datasets/master/electricity_consumption/Electric_Production.csv"

In [4]:
# Gerar o data frame 
df = pd.read_csv(df_path)

# Ver o data frame
df.head()

,DATE,Value
0,01-01-1985,72.5052
1,02-01-1985,70.6720
2,03-01-1985,62.4502
3,04-01-1985,57.4714
4,05-01-1985,55.3151


In [ ]:
# Ver as info do data frame 
# O campo DATE esta no formato object e precisa ser data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DATE    397 non-null    object 
 1   Value   397 non-null    float64
dtypes: float64(1), object(1)
memory usage: 6.3+ KB


In [ ]:
# Criar uma nova coluna no formato de data 

df.index = pd.to_datetime(df.DATE, format = "%m-%d-%Y")

In [ ]:
# Dropar a coluna date

df.drop("DATE", inplace=True, axis=1)

In [ ]:
# Ver como ficou os dados 

df.head()

,Value
DATE,
1985-01-01,72.5052
1985-02-01,70.6720
1985-03-01,62.4502
1985-04-01,57.4714
1985-05-01,55.3151


In [18]:
df.loc['1985-05-01']

Value    55.3151
Name: 1985-05-01 00:00:00, dtype: float64

#### Aula 2 - Utilizando Datas como Índice

#### Aula 3 - Bibliotecas Fundamentais em Time Series Problem

#### Aula 4 - Análise Exploratória de Dados (EDA)

#### Aula 5 - Modelando e Realizando o Forecasting